In [1]:
from langchain_core.documents import Document

In [2]:


doc = Document(
    metadata={
        "source": "example.txt",
        "pages": 1,
        "author": "Krish Naik",
        "date_created": "2025-01-01"
    },
    page_content="this is the main text content I am using to create RAG"
)

doc

Document(metadata={'source': 'example.txt', 'pages': 1, 'author': 'Krish Naik', 'date_created': '2025-01-01'}, page_content='this is the main text content I am using to create RAG')

In [3]:
import os
os. makedirs("../data/text_files", exist_ok=True)

In [4]:
sample_texts = {
    "../data/text_files/finadapt_intro.txt": """
FinAdapt – India’s First Agentic-AI Powered Financial Guidance System for Gig Workers

FinAdapt is a next-generation financial empowerment platform designed exclusively for India’s rapidly growing gig economy. 
In a world where freelancers, delivery partners, cab drivers, creators, tutors, technicians, and part-time earners struggle 
with irregular income and zero traditional financial support, FinAdapt acts as a dedicated, intelligent financial mentor.

Unlike typical budgeting apps that only track expenses, FinAdapt uses agentic AI––a system capable of autonomous reasoning, 
planning, and multi-step decision making––to understand the user’s income patterns, financial habits, and long-term goals. 
It then creates real-time, personalized strategies for savings, investments, expense management, loans, taxes, and 
overall financial health.

FinAdapt is built on the belief that **gig workers deserve the same financial clarity, security, and opportunity as 
full-time employees**, even if their income is unstable.

Key Capabilities of FinAdapt:
- **Dynamic budgeting engine** that adapts automatically to fluctuating income levels every week
- **AI-generated smart savings goals** for essentials, emergencies, and personal aspirations
- **Cashflow forecasting** that predicts upcoming shortages or surplus using the user’s historical spending behavior
- **Automated tax assistant** that calculates tax liability, eligible deductions, and compliance tasks for gig workers
- **Intelligent expense categorization** using AI-based receipt reading and transaction analysis
- **Personalized loan readiness score** to help gig workers understand eligibility and reduce financial risk
- **Insurance guidance** that explains health, life, and accident coverage in extremely simple words
- **Educational micro-modules** that teach users financial literacy in easy regional-language content
- **Real-time agentic AI mentor** that can plan, prioritize, compare, calculate, and recommend financial decisions like a human advisor
- **Goal-based financial planning** for buying a bike, phone, home, education, or starting a business

Vision and Impact:
FinAdapt aims to become the financial backbone of India’s 100+ million gig and informal workers. The platform is not 
just a tool—it is a safety net, a support system, and a long-term financial growth partner. By combining agentic AI with 
deep financial workflows, FinAdapt ensures that every gig worker gains access to structured financial planning without 
needing prior knowledge or stable monthly income.

With FinAdapt, gig workers finally receive:
- More control over cashflow
- Better financial decisions
- Reduced stress during low-income months
- Improved long-term financial stability
- A stronger path toward savings, investments, and creditworthiness

FinAdapt isn’t just another fintech project—it is a mission to build financial confidence, bridge opportunity gaps, 
and bring financial dignity to millions of independent workers across the country.
"""
}

for filepath, content in sample_texts.items():
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content)

print("Sample text files created!")

Sample text files created!


In [5]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("../data/text_files/finadapt_intro.txt", encoding="utf-8")
documents = loader.load()

print("Loaded Documents:")
print(documents)

Loaded Documents:
[Document(metadata={'source': '../data/text_files/finadapt_intro.txt'}, page_content='\nFinAdapt – India’s First Agentic-AI Powered Financial Guidance System for Gig Workers\n\nFinAdapt is a next-generation financial empowerment platform designed exclusively for India’s rapidly growing gig economy. \nIn a world where freelancers, delivery partners, cab drivers, creators, tutors, technicians, and part-time earners struggle \nwith irregular income and zero traditional financial support, FinAdapt acts as a dedicated, intelligent financial mentor.\n\nUnlike typical budgeting apps that only track expenses, FinAdapt uses agentic AI––a system capable of autonomous reasoning, \nplanning, and multi-step decision making––to understand the user’s income patterns, financial habits, and long-term goals. \nIt then creates real-time, personalized strategies for savings, investments, expense management, loans, taxes, and \noverall financial health.\n\nFinAdapt is built on the belief 

In [6]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

# Load all text files from the directory
dir_loader = DirectoryLoader(
    "../data/text_files",            # directory path
    glob="**/*.txt",                # pattern to match .txt files
    loader_cls=TextLoader,          # loader to use for each file
    loader_kwargs={"encoding": "utf-8"},
)

documents = dir_loader.load()

print("Loaded documents:")
print(documents)

Loaded documents:
[Document(metadata={'source': '../data/text_files/finadapt_intro.txt'}, page_content='\nFinAdapt – India’s First Agentic-AI Powered Financial Guidance System for Gig Workers\n\nFinAdapt is a next-generation financial empowerment platform designed exclusively for India’s rapidly growing gig economy. \nIn a world where freelancers, delivery partners, cab drivers, creators, tutors, technicians, and part-time earners struggle \nwith irregular income and zero traditional financial support, FinAdapt acts as a dedicated, intelligent financial mentor.\n\nUnlike typical budgeting apps that only track expenses, FinAdapt uses agentic AI––a system capable of autonomous reasoning, \nplanning, and multi-step decision making––to understand the user’s income patterns, financial habits, and long-term goals. \nIt then creates real-time, personalized strategies for savings, investments, expense management, loans, taxes, and \noverall financial health.\n\nFinAdapt is built on the belief 

In [7]:
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
import os

# Choose one PDF loader
pdf_loader_cls = PyMuPDFLoader  # or PyPDFLoader

# Directory containing PDF files
pdf_directory = "../data/pdf"

# Ensure directory exists
if not os.path.exists(pdf_directory):
    os.makedirs(pdf_directory)

# Load all PDFs
documents = []

for root, dirs, files in os.walk(pdf_directory):
    for file in files:
        if file.lower().endswith(".pdf"):
            file_path = os.path.join(root, file)
            loader = pdf_loader_cls(file_path)
            docs = loader.load()
            documents.extend(docs)

print(f"Total PDF documents loaded: {len(documents)}")


Total PDF documents loaded: 4


In [8]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# Initialize the text splitter with better parameters for financial content
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,       # Reduced from 1500 to keep related content together
    chunk_overlap=200,     # Increased overlap for better context
    separators=["\n\n", "\n", "(?<=\\. )", " ", ""],  # Added lookbehind for sentence splitting
    keep_separator=True,   # Keep separators in the text
    length_function=len,   # Use character count
    is_separator_regex=True  # Enable regex for separators
)

# Enhanced chunking with metadata preservation
def chunk_documents(documents):
    chunked_docs = []
    
    for doc in documents:
        # Preserve original metadata
        metadata = getattr(doc, 'metadata', {}).copy()
        
        # Split the document
        chunks = text_splitter.split_documents([doc])
        
        # Add chunk-specific metadata
        for i, chunk in enumerate(chunks):
            # Preserve original metadata
            chunk.metadata.update(metadata)
            
            # Add chunk-specific metadata
            chunk.metadata.update({
                "chunk_index": i,
                "total_chunks": len(chunks),
                "chunk_type": "text"  # Could be 'table', 'figure', etc. if you add that logic
            })
            
            # Clean up the chunk content
            chunk.page_content = chunk.page_content.strip()
            
            # Only add non-empty chunks
            if chunk.page_content:
                chunked_docs.append(chunk)
    
    return chunked_docs

# Apply the chunking to your documents
chunked_documents = chunk_documents(documents)

print(f"Original documents: {len(documents)}")
print(f"Chunked documents: {len(chunked_documents)}")

# Verify chunk quality
print("\nSample chunks:")
for i, chunk in enumerate(chunked_documents[:2]):  # Show first 2 chunks
    print(f"\n--- Chunk {i+1} ---")
    print("Content:", chunk.page_content[:200] + "..." if len(chunk.page_content) > 200 else chunk.page_content)
    print("Metadata:", {k: v for k, v in chunk.metadata.items() if not k.startswith('_')})
    print("-" * 80)

Original documents: 4
Chunked documents: 11

Sample chunks:

--- Chunk 1 ---
Content: # Anil Kapoor's 2025 Comprehensive Financial Report 
 
**Generated by FinAdapt Agentic-AI on November 27, 2025**   
**User Profile:** Anil Kapoor (anil12@gmail.com)   
**Occupation:** Gig Worker | Pla...
Metadata: {'producer': 'Skia/PDF m144 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '../data/pdf/data.pdf', 'file_path': '../data/pdf/data.pdf', 'total_pages': 4, 'format': 'PDF 1.4', 'title': 'Untitled document', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0, 'chunk_index': 0, 'total_chunks': 3, 'chunk_type': 'text'}
--------------------------------------------------------------------------------

--- Chunk 2 ---
Content: | **Net Savings** | 159,600     | 30%        | 
| **Avg Monthly Income** | 44,583 | -          | 
| **Avg Monthly Savings** | 13,300 | -          | 
 
### Loan & Investment Snapshot 
- **

In [10]:
from sentence_transformers import SentenceTransformer
import numpy as np
from typing import List, Optional, Union
import logging
from pathlib import Path

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class EmbeddingManager:
    """
    Document embedding manager using SentenceTransformer.
    """

    def __init__(
        self,
        model_name: str = "all-MiniLM-L6-v2",
        device: Optional[str] = None,
        cache_folder: Optional[Union[str, Path]] = None
    ):
        self.model_name = model_name
        self.device = device
        self.cache_folder = str(cache_folder) if cache_folder else None
        self.model = None
        self.embedding_dimension = None
        self._load_model()
    
    def _load_model(self) -> None:
        """Load the SentenceTransformer model."""
        try:
            logger.info(f"Loading model: {self.model_name}")
            self.model = SentenceTransformer(
                self.model_name,
                device=self.device,
                cache_folder=self.cache_folder
            )
            self.embedding_dimension = self.model.get_sentence_embedding_dimension()
            logger.info(f"Model loaded. Embedding dimension: {self.embedding_dimension}")
        except Exception as e:
            logger.error(f"Failed to load model {self.model_name}: {e}")
            raise
    
    def get_embedding(self, text: str, normalize: bool = True) -> np.ndarray:
        """Generate embedding for a single text."""
        if not text:
            return np.zeros(self.embedding_dimension, dtype=np.float32)
        return self.model.encode(text, convert_to_numpy=True, normalize_embeddings=normalize)

    def get_embeddings(self, texts: List[str], batch_size: int = 32, show_progress_bar: bool = True, normalize: bool = True) -> np.ndarray:
        """Generate embeddings for a list of texts."""
        if not texts:
            return np.empty((0, self.embedding_dimension), dtype=np.float32)
        return self.model.encode(
            texts,
            batch_size=batch_size,
            show_progress_bar=show_progress_bar,
            convert_to_numpy=True,
            normalize_embeddings=normalize
        )
    
    def embed_query(self, text: str) -> np.ndarray:
        """Wrapper for query embedding (used by RAGRetriever)."""
        return self.get_embedding(text)

    def get_embedding_dimension(self) -> int:
        return self.embedding_dimension or 0

In [11]:
import os
import uuid
import numpy as np
import chromadb
from typing import List, Any

class VectorStore:
    """
    Manages document embeddings in a ChromaDB vector store
    """

    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents: {self.collection.count()}")
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """Add documents and their embeddings to the vector store"""
        if len(documents) != len(embeddings):
            raise ValueError("Documents and embeddings length mismatch")

        ids, metadatas, docs_text, embeddings_list = [], [], [], []

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            metadata = dict(getattr(doc, 'metadata', {}))
            metadata['doc_index'] = i
            metadata['content_length'] = len(getattr(doc, 'page_content', ''))
            metadatas.append(metadata)

            docs_text.append(getattr(doc, 'page_content', ''))
            embeddings_list.append(embedding.tolist() if isinstance(embedding, np.ndarray) else embedding)

        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=docs_text
            )
            print(f"Added {len(documents)} documents. Total now: {self.collection.count()}")
        except Exception as e:
            print(f"Error adding documents: {e}")
            raise

    def search(self, query_embedding, top_k: int = 5):
        """Search the vector store using query embedding"""
        try:
            # Convert numpy array to list if needed
            if hasattr(query_embedding, 'tolist'):
                query_embedding = query_embedding.tolist()

            results = self.collection.query(
                query_embeddings=[query_embedding],
                n_results=top_k
            )

            # Format results
            formatted_results = []
            if results and 'metadatas' in results and 'documents' in results:
                for meta, content in zip(results['metadatas'][0], results['documents'][0]):
                    formatted_results.append({
                        'content': content,
                        'metadata': meta
                    })

            return formatted_results
        except Exception as e:
            print(f"Search failed: {e}")
            return []

In [12]:
# Make sure chunked_documents is already created
# chunked_documents = [Document(page_content="...", metadata={...}), ...]

# Prepare text content for embeddings
texts = [doc.page_content for doc in chunked_documents]

# --- FIX: create an instance of EmbeddingManager ---
embedding_manager = EmbeddingManager(model_name="all-MiniLM-L6-v2")

# Generate embeddings
embeddings = embedding_manager.get_embeddings(texts)

print(f"Generated embeddings for {len(texts)} chunks.")

# Check example embedding
if embeddings is not None and len(embeddings) > 0:
    print(f"Shape of first embedding (example): {embeddings[0].shape}")
else:
    print("No embeddings")

INFO:__main__:Loading model: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:__main__:Model loaded. Embedding dimension: 384


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings for 11 chunks.
Shape of first embedding (example): (384,)


In [13]:
from typing import List, Any
import traceback

class RAGRetriever:
    """Retrieve top documents from vector store given a query"""

    def __init__(self, vector_store, embedding_manager):
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5) -> List[Any]:
        try:
            query_embedding = self.embedding_manager.embed_query(query)
        except Exception as e:
            print(f"[ERROR] Failed to generate query embedding: {e}")
            traceback.print_exc()
            return []

        try:
            results = self.vector_store.search(query_embedding, top_k=top_k)
            return results
        except Exception as e:
            print(f"[ERROR] Vector store search failed: {e}")
            traceback.print_exc()
            return []

In [14]:
# 1️⃣ Initialize embedding manager and vector store
embedding_manager = EmbeddingManager("all-MiniLM-L6-v2")
vector_store = VectorStore("finadapt_docs", "../data/vector_store")

# 2️⃣ Prepare documents (must exist)
# Example: chunked_documents = [{'page_content': "text here", 'metadata': {...}}, ...]
document_contents = [doc.page_content for doc in chunked_documents]
document_embeddings = embedding_manager.get_embeddings(document_contents)

# 3️⃣ Add to vector store
vector_store.add_documents(chunked_documents, document_embeddings)

# 4️⃣ Initialize retriever
document_retriever = RAGRetriever(vector_store, embedding_manager)

# 5️⃣ Retrieve
search_results = document_retriever.retrieve(
    query="What is FinAdapt's core mission?",
    top_k=5
)

# 6️⃣ Display results
for idx, doc in enumerate(search_results, 1):
    print(f"\n--- Document {idx} ---")
    print("Content:", doc['content'][:200] + "..." if len(doc['content']) > 200 else doc['content'])
    print("Metadata:", doc['metadata'])
    print("-" * 50)

INFO:__main__:Loading model: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:__main__:Model loaded. Embedding dimension: 384
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


Vector store initialized. Collection: finadapt_docs
Existing documents: 0


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Added 11 documents. Total now: 11


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


--- Document 1 ---
Content: # Anil Kapoor's 2025 Comprehensive Financial Report 
 
**Generated by FinAdapt Agentic-AI on November 27, 2025**   
**User Profile:** Anil Kapoor (anil12@gmail.com)   
**Occupation:** Gig Worker | Pla...
Metadata: {'modDate': '', 'chunk_type': 'text', 'doc_index': 0, 'producer': 'Skia/PDF m144 Google Docs Renderer', 'format': 'PDF 1.4', 'page': 0, 'content_length': 989, 'chunk_index': 0, 'subject': '', 'creationDate': '', 'moddate': '', 'total_pages': 4, 'creationdate': '', 'trapped': '', 'author': '', 'creator': '', 'title': 'Untitled document', 'file_path': '../data/pdf/data.pdf', 'keywords': '', 'total_chunks': 3, 'source': '../data/pdf/data.pdf'}
--------------------------------------------------

--- Document 2 ---
Content: | **Net Savings** | 159,600     | 30%        | 
| **Avg Monthly Income** | 44,583 | -          | 
| **Avg Monthly Savings** | 13,300 | -          | 
 
### Loan & Investment Snapshot 
- **Active Loan:*...
Metadata: {'total_chunks': 3

### Connetion with llm

In [15]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import Optional, List, Dict, Any

load_dotenv()

False

In [16]:
# ---------------------------
# 1. Initialize Groq LLM
# ---------------------------

from groq import Groq
from langchain_groq import ChatGroq   # if using LangChain wrapper

groq_api_key = "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"   # never hardcode real keys

llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama-3.3-70b-versatile",   # valid model
    temperature=0.1,
    max_tokens=1024
)

# ---------------------------
# 2. Simple RAG Function
# ---------------------------

# ---------------------------
# 2. Simple RAG Function (Updated)
# ---------------------------

def rag_simple(query, retriever, llm, top_k=5):
    """
    Enhanced RAG function that provides detailed, explanatory answers.
    Combines retrieved context with general knowledge for comprehensive responses.
    """
    # 1️⃣ Expand query for better retrieval
    expanded_queries = [query]
    if any(term in query.lower() for term in ["expense", "categorization", "financial", "feature", "how", "what", "explain"]):
        expanded_queries.extend([
            f"Detailed explanation of {query}",
            f"How FinAdapt implements {query}",
            f"Technical details about {query} in FinAdapt"
        ])

    # 2️⃣ Retrieve documents with expanded queries
    all_results = []
    for q in expanded_queries:
        try:
            results = retriever.retrieve(q, top_k=top_k)
            all_results.extend([r for r in results if r not in all_results])
        except Exception as e:
            continue

    # 3️⃣ Build context with source tracking
    context_parts = []
    for i, doc in enumerate(all_results[:top_k], 1):
        content = doc.get('content', '').strip()
        if content:
            context_parts.append(f"--- Source {i} ---\n{content}\n")

    context = "\n".join(context_parts) if context_parts else "No specific context found."

    # 4️⃣ Enhanced prompt for explanatory responses
    prompt = f"""You are a knowledgeable financial technology expert explaining FinAdapt's features.

Context from documentation:
{context}

Question: {query}

Please provide a detailed, informative response that:
1. Directly answers the question based on the context
2. If context is limited, supplement with general industry knowledge
3. Explain how the feature works and its benefits
4. Provide examples or use cases where applicable
5. Keep the explanation clear and professional

Structure your response with:
- A clear opening statement
- Key points in bullet format
- A brief conclusion

Detailed response:"""

    # 5️⃣ Generate and process the response
    try:
        response = llm.invoke(input=prompt)
        answer = response.content if hasattr(response, 'content') else str(response)
        
        # 6️⃣ Ensure the response is sufficiently detailed
        if len(answer.split()) < 50:  # If response is too short
            answer = f"""{answer}

            Let me elaborate further. In financial technology platforms like FinAdapt, {query.lower().replace('?', '')} typically involves:

            - Advanced algorithms that analyze transaction patterns
            - Machine learning models that improve over time
            - User-friendly interfaces for easy management
            - Integration with banking and payment systems

            This comprehensive approach ensures users get maximum value from the platform's features."""
        
        return answer

    except Exception as e:
        # Fallback with general knowledge
        return f"""I'll provide a detailed explanation based on general knowledge of financial platforms:

        Regarding {query}, modern financial platforms like FinAdapt typically offer:

        - Intelligent systems that automatically categorize transactions
        - Machine learning algorithms that learn from user behavior
        - Customizable categories and rules for personalization
        - Detailed reporting and analytics

        These features help users:
        - Better understand their spending habits
        - Save time on financial management
        - Make more informed financial decisions
        - Achieve their financial goals more effectively

        While I don't have the specific implementation details for FinAdapt, this reflects industry standards for such features in leading financial platforms."""

# Example usage:
# answer = rag_simple("How does expense categorization work in FinAdapt?", document_retriever, llm)
# print(answer)

In [18]:
answer = rag_simple(
    "What is the total income and expenses for Anil Kapoor in January 2025?",
    document_retriever,  # Your retriever instance
    llm,
    top_k=5
)
print(answer)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


To determine Anil Kapoor's total income and expenses for January 2025, we can refer to the comprehensive financial report generated by FinAdapt Agentic-AI. According to the report, the total income and expenses for January 2025 are as follows:

The total income for January 2025 is ₹42,000, and the total expenses are ₹26,850, resulting in savings of ₹15,150. 

Here are the key points related to Anil Kapoor's financial data for January 2025:
* Total income: ₹42,000
* Total expenses: ₹26,850
* Savings: ₹15,150
* Key insight: Fuel nearing budget—monitor, indicating that Anil Kapoor should keep a close eye on his fuel expenses to avoid exceeding his budget.
* The data is sourced from FinAdapt's dashboard, which provides a comprehensive overview of Anil Kapoor's financial transactions, including income, expenses, and savings.

FinAdapt's feature of tracking income and expenses works by aggregating data from various sources, including daily gigs, weekly contracts, and other financial transact